NOTE: Some of the results here will not necessarily match with the results from our Written Report due to random number generation with computation time and different test splits affecting accuracy but should the differences should be negligble.

Installing all important libraries and packages

In [ ]:
!pip install imblearn

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, classification_report
import time
from sklearn.decomposition import PCA
from collections import Counter
from sklearn.model_selection import train_test_split, StratifiedKFold
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU, Input, Lambda, Layer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K

full thing with PCA


In [ ]:
# what does this do :
start_time = time.time()

data = pd.read_csv("METABRIC_RNA_Mutation.csv")

columns_to_drop = [
    "age_at_diagnosis", "tumor_other_histologic_subtype", "inferred_menopausal_state",
    "mutation_count", "hormone_therapy", "patient_id", "cellularity",
    "type_of_breast_surgery", "cohort", "chemotherapy", "tumor_size",
    "hormone_therapy", "menopausal_state", "overall_survival_months",
    "overall_survival", "death_from_cancer", "tumor_stage", "er_status",
    "pam50_+_claudin-low_subtype", "pr_status", "er_status_measured_by_ihc",
    "nottingham_prognostic_index", "tumor_stage", "death_from_cancer",
    "integrative_cluster", "her2_status_measured_by_snp6", "her2_status",
    "radio_therapy", "3-gene_classifier_subtype",   "oncotree_code"
]

data = data.drop(columns=columns_to_drop, errors='ignore')

print(data["cancer_type_detailed"].value_counts())

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data["cancer_type_detailed"])

X = pd.get_dummies(data.drop(columns=["cancer_type_detailed"]), drop_first=True)

imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

print(X_scaled.shape)

lasso = Lasso(alpha=0.01)
lasso.fit(X_scaled, y)

selected_featuresLasso = X.columns[(lasso.coef_ != 0)]

print("Selected features:", selected_featuresLasso)


X=X[selected_featuresLasso]

print(X.shape)

X_selected = X[selected_featuresLasso]

imputer = SimpleImputer(strategy='mean')
X_selected_imputed = imputer.fit_transform(X_selected)

pca = PCA(n_components=0.95)  # Retain 95% of the variance
X_pca = pca.fit_transform(X_selected_imputed)

# Output the number of components and transformed data shape
print("Number of PCA components:", pca.n_components_)
print("Shape of PCA-transformed data:", X_pca.shape)


class_counts = Counter(y)
single_instance_classes = [cls for cls, count in class_counts.items() if count < 2]

# Create a mask to filter out these classes
mask = ~np.isin(y, single_instance_classes)
X_pca_filtered = X_pca[mask]
y_filtered = y[mask]

# Check the new class distribution after filtering
print("Class distribution after filtering:", Counter(y_filtered))

# Now split the filtered data


X_train, X_test, y_train, y_test = train_test_split(X_pca_filtered, y_filtered, test_size=0.3, stratify=y_filtered, random_state=42)

# Apply SMOTE to the training set
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Check the new class distribution after SMOTE
print("Class distribution after SMOTE:", Counter(y_train_balanced))


# Parameters
num_folds = 5
batch_size = 32
epochs = 20
learning_rate = 0.0005

# Re-encode y_train_balanced to ensure sequential class labels
label_encoder = LabelEncoder()
y_train_balanced_encoded = label_encoder.fit_transform(y_train_balanced)

# Define the K-fold Cross Validator
kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

# Array to store fold results
fold_accuracies = []
fold_losses = []

# Determine input shape dynamically
input_shape = X_train_balanced.shape[1]
num_classes = len(np.unique(y_train_balanced_encoded))  # Determine number of classes

for fold, (train_idx, val_idx) in enumerate(kfold.split(X_train_balanced, y_train_balanced_encoded)):
    print(f"Training fold {fold+1}/{num_folds}...")

    # Split data
    X_train_fold, X_val_fold = X_train_balanced[train_idx], X_train_balanced[val_idx]
    y_train_fold, y_val_fold = y_train_balanced_encoded[train_idx], y_train_balanced_encoded[val_idx]

    # Define the model (redefine for each fold to reset weights)
    model = Sequential([
        Dense(128, activation='relu', kernel_regularizer=l2(0.001), input_shape=(input_shape,)),
        Dropout(0.2),
        Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
        Dropout(0.2),
        Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
        Dropout(0.2),
        Dense(num_classes, activation='softmax')  # Output layer matches re-encoded classes
    ])

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    history = model.fit(X_train_fold, y_train_fold, epochs=epochs, batch_size=batch_size, validation_data=(X_val_fold, y_val_fold), verbose=1)

    # Evaluate the model
    val_loss, val_accuracy = model.evaluate(X_val_fold, y_val_fold, verbose=0)
    fold_losses.append(val_loss)
    fold_accuracies.append(val_accuracy)
    print(f"Fold {fold+1} - Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}")

# Report cross-validation results
print("\nCross-validation results:")
print(f"Mean Validation Loss: {np.mean(fold_losses):.4f} ± {np.std(fold_losses):.4f}")
print(f"Mean Validation Accuracy: {np.mean(fold_accuracies):.4f} ± {np.std(fold_accuracies):.4f}")

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Get predicted class labels


accuracy = accuracy_score(y_test, y_pred_classes)
print("Test Accuracy:", accuracy)


f1 = f1_score(y_test, y_pred_classes, average='weighted')
print("Test F1 Score:", f1)

print("\nClassification Report:\n", classification_report(y_test, y_pred_classes))


end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")


SMOTE Only

In [ ]:
start_time = time.time()

# Load the data
data = pd.read_csv("METABRIC_RNA_Mutation.csv")

# Define columns to drop
columns_to_drop = [
    "age_at_diagnosis", "tumor_other_histologic_subtype", "inferred_menopausal_state",
    "mutation_count", "hormone_therapy", "patient_id", "cellularity",
    "type_of_breast_surgery", "cohort", "chemotherapy", "tumor_size",
    "hormone_therapy", "menopausal_state", "overall_survival_months",
    "overall_survival", "death_from_cancer", "tumor_stage", "er_status",
    "pam50_+_claudin-low_subtype", "pr_status", "er_status_measured_by_ihc",
    "nottingham_prognostic_index", "tumor_stage", "death_from_cancer",
    "integrative_cluster", "her2_status_measured_by_snp6", "her2_status",
    "radio_therapy", "3-gene_classifier_subtype", "oncotree_code"
]

# Drop specified columns and encode the target variable
data = data.drop(columns=columns_to_drop, errors='ignore')
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data["cancer_type_detailed"])

# One-hot encode features and impute missing values
X = pd.get_dummies(data.drop(columns=["cancer_type_detailed"]), drop_first=True)
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

print(X_scaled.shape)

# Filter out classes with fewer than two instances
class_counts = Counter(y)
single_instance_classes = [cls for cls, count in class_counts.items() if count < 2]
mask = ~np.isin(y, single_instance_classes)
X_filtered = X_scaled[mask]
y_filtered = y[mask]

# Split data and apply SMOTE to the training set
X_train, X_test, y_train, y_test = train_test_split(X_filtered, y_filtered, test_size=0.3, stratify=y_filtered, random_state=42)
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

print("Class distribution after SMOTE:", Counter(y_train_balanced))

# Re-encode y_train_balanced and y_test to ensure sequential labels
label_encoder = LabelEncoder()
y_train_balanced_encoded = label_encoder.fit_transform(y_train_balanced)
y_test_encoded = label_encoder.transform(y_test)
num_classes = len(np.unique(y_train_balanced_encoded))

# Set cross-validation and neural network parameters
num_folds = 5
batch_size = 32
epochs = 20
learning_rate = 0.0005

# Initialize arrays to store fold results
fold_accuracies = []
fold_losses = []

# Define Stratified K-Fold Cross Validator
kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
input_shape = X_train_balanced.shape[1]

# Cross-validation training
for fold, (train_idx, val_idx) in enumerate(kfold.split(X_train_balanced, y_train_balanced_encoded)):
    print(f"Training fold {fold+1}/{num_folds}...")

    X_train_fold, X_val_fold = X_train_balanced[train_idx], X_train_balanced[val_idx]
    y_train_fold, y_val_fold = y_train_balanced_encoded[train_idx], y_train_balanced_encoded[val_idx]

    # Define and compile the neural network model
    model = Sequential([
        Dense(128, activation='relu', kernel_regularizer=l2(0.001), input_shape=(input_shape,)),
        Dropout(0.2),
        Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
        Dropout(0.2),
        Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
        Dropout(0.2),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train and evaluate the model
    model.fit(X_train_fold, y_train_fold, epochs=epochs, batch_size=batch_size, validation_data=(X_val_fold, y_val_fold), verbose=1)
    val_loss, val_accuracy = model.evaluate(X_val_fold, y_val_fold, verbose=0)
    fold_losses.append(val_loss)
    fold_accuracies.append(val_accuracy)
    print(f"Fold {fold+1} - Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}")

# Report cross-validation results
print("\nCross-validation results:")
print(f"Mean Validation Loss: {np.mean(fold_losses):.4f} ± {np.std(fold_losses):.4f}")
print(f"Mean Validation Accuracy: {np.mean(fold_accuracies):.4f} ± {np.std(fold_accuracies):.4f}")

# Final evaluation on the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

accuracy = accuracy_score(y_test_encoded, y_pred_classes)
print("Test Accuracy:", accuracy)

f1 = f1_score(y_test_encoded, y_pred_classes, average='weighted')
print("Test F1 Score:", f1)

print("\nClassification Report:\n", classification_report(y_test_encoded, y_pred_classes))


# Calculate the confusion matrix
conf_matrix = confusion_matrix(y_test_encoded, y_pred_classes)

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")


Lasso and SMOTE Only

In [ ]:
start_time = time.time()

# Step 1: Data Loading and Cleaning
data = pd.read_csv("METABRIC_RNA_Mutation.csv", low_memory=False)
columns_to_drop = ["age_at_diagnosis", "tumor_other_histologic_subtype", "inferred_menopausal_state",
                   "mutation_count", "hormone_therapy", "patient_id", "cellularity",
                   "type_of_breast_surgery", "cohort", "chemotherapy", "tumor_size",
                   "hormone_therapy", "menopausal_state", "overall_survival_months",
                   "overall_survival", "death_from_cancer", "tumor_stage", "er_status",
                   "pam50_+_claudin-low_subtype", "pr_status", "er_status_measured_by_ihc",
                   "nottingham_prognostic_index", "tumor_stage", "death_from_cancer",
                   "integrative_cluster", "her2_status_measured_by_snp6", "her2_status",
                   "radio_therapy", "3-gene_classifier_subtype", "oncotree_code"]
data = data.drop(columns=columns_to_drop, errors='ignore')
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data["cancer_type_detailed"])

# Step 2: One-Hot Encoding & Imputation
X = pd.get_dummies(data.drop(columns=["cancer_type_detailed"]), drop_first=True)
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Step 3: Feature Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Step 4: Feature Selection with Lasso
lasso = Lasso(alpha=0.01)
lasso.fit(X_scaled, y)
selected_featuresLasso = X.columns[(lasso.coef_ != 0)]
X = pd.DataFrame(X_scaled, columns=X.columns)[selected_featuresLasso]

# Step 5: Class Filtering
class_counts = Counter(y)
mask = ~np.isin(y, [cls for cls, count in class_counts.items() if count < 2])
X, y = X[mask], y[mask]

# Step 6: Data Splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# Step 7: Class Balancing with SMOTE
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Re-encode y_train_balanced to ensure sequential class labels
label_encoder = LabelEncoder()
y_train_balanced = label_encoder.fit_transform(y_train_balanced)

# Ensure X_train_balanced is in NumPy format
if isinstance(X_train_balanced, pd.DataFrame):
    X_train_balanced = X_train_balanced.to_numpy()

# Step 8: Cross-Validation Training
num_folds = 5
batch_size = 32
epochs = 20
learning_rate = 0.0005
input_shape = X_train_balanced.shape[1]
num_classes = len(np.unique(y_train_balanced))  # Determine number of classes
kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
fold_accuracies, fold_losses = [], []

for train_idx, val_idx in kfold.split(X_train_balanced, y_train_balanced):
    X_train_fold, X_val_fold = X_train_balanced[train_idx], X_train_balanced[val_idx]
    y_train_fold, y_val_fold = y_train_balanced[train_idx], y_train_balanced[val_idx]

    model = Sequential([
        Input(shape=(input_shape,)),  # Use Input layer as first layer
        Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
        Dropout(0.2),
        Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
        Dropout(0.2),
        Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
        Dropout(0.2),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train_fold, y_train_fold, epochs=epochs, batch_size=batch_size, validation_data=(X_val_fold, y_val_fold), verbose=1)
    val_loss, val_accuracy = model.evaluate(X_val_fold, y_val_fold, verbose=0)
    fold_losses.append(val_loss)
    fold_accuracies.append(val_accuracy)

print("\nCross-validation results:")
print(f"Mean Validation Loss: {np.mean(fold_losses):.4f} ± {np.std(fold_losses):.4f}")
print(f"Mean Validation Accuracy: {np.mean(fold_accuracies):.4f} ± {np.std(fold_accuracies):.4f}")

# Step 9: Final Model Evaluation
y_pred = np.argmax(model.predict(X_test), axis=1)
print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("Test F1 Score:", f1_score(y_test, y_pred, average='weighted'))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")


Autoencoder full

In [ ]:
start_time = time.time()

# Load the data
data = pd.read_csv("METABRIC_RNA_Mutation.csv")

# Data Cleaning: Drop specified columns and encode target variable
columns_to_drop = [
    "age_at_diagnosis", "tumor_other_histologic_subtype", "inferred_menopausal_state",
    "mutation_count", "hormone_therapy", "patient_id", "cellularity",
    "type_of_breast_surgery", "cohort", "chemotherapy", "tumor_size",
    "hormone_therapy", "menopausal_state", "overall_survival_months",
    "overall_survival", "death_from_cancer", "tumor_stage", "er_status",
    "pam50_+_claudin-low_subtype", "pr_status", "er_status_measured_by_ihc",
    "nottingham_prognostic_index", "tumor_stage", "death_from_cancer",
    "integrative_cluster", "her2_status_measured_by_snp6", "her2_status",
    "radio_therapy", "3-gene_classifier_subtype", "oncotree_code"
]
data = data.drop(columns=columns_to_drop, errors='ignore')
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data["cancer_type_detailed"])

# One-Hot Encoding & Imputation
X = pd.get_dummies(data.drop(columns=["cancer_type_detailed"]), drop_first=True)
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Feature Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Feature Selection with Lasso
lasso = Lasso(alpha=0.01)
lasso.fit(X_scaled, y)
selected_featuresLasso = X.columns[(lasso.coef_ != 0)]
X_selected = X_scaled[:, lasso.coef_ != 0]

# Autoencoder for Dimensionality Reduction
input_dim = X_selected.shape[1]
encoding_dim = 50  # Set the encoding dimension (adjust based on desired reduction level)

input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation='relu')(input_layer)
decoder = Dense(input_dim, activation='sigmoid')(encoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)
autoencoder.compile(optimizer='adam', loss='mse')

# Train the autoencoder
autoencoder.fit(X_selected, X_selected, epochs=50, batch_size=32, shuffle=True, validation_split=0.2, verbose=0)

# Extract the encoder part to get the compressed representation
encoder_model = Model(inputs=input_layer, outputs=encoder)
X_encoded = encoder_model.predict(X_selected)

# Class Filtering: Remove classes with fewer than 2 instances
class_counts = Counter(y)
single_instance_classes = [cls for cls, count in class_counts.items() if count < 2]
mask = ~np.isin(y, single_instance_classes)
X_filtered = X_encoded[mask]
y_filtered = y[mask]

# Data Splitting
X_train, X_test, y_train, y_test = train_test_split(X_filtered, y_filtered, test_size=0.3, stratify=y_filtered, random_state=42)

# Class Balancing with SMOTE
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Cross-Validation Training
num_folds = 5
batch_size = 32
epochs = 20
learning_rate = 0.0005
y_train_balanced_encoded = LabelEncoder().fit_transform(y_train_balanced)
input_shape = X_train_balanced.shape[1]
num_classes = len(np.unique(y_train_balanced_encoded))

kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
fold_accuracies = []
fold_losses = []

for fold, (train_idx, val_idx) in enumerate(kfold.split(X_train_balanced, y_train_balanced_encoded)):
    print(f"Training fold {fold+1}/{num_folds}...")
    X_train_fold, X_val_fold = X_train_balanced[train_idx], X_train_balanced[val_idx]
    y_train_fold, y_val_fold = y_train_balanced_encoded[train_idx], y_train_balanced_encoded[val_idx]

    model = Sequential([
        Dense(128, activation='relu', kernel_regularizer=l2(0.001), input_shape=(input_shape,)),
        Dropout(0.2),
        Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
        Dropout(0.2),
        Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
        Dropout(0.2),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train_fold, y_train_fold, epochs=epochs, batch_size=batch_size, validation_data=(X_val_fold, y_val_fold), verbose=1)
    val_loss, val_accuracy = model.evaluate(X_val_fold, y_val_fold, verbose=0)
    fold_losses.append(val_loss)
    fold_accuracies.append(val_accuracy)
    print(f"Fold {fold+1} - Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}")

# Report cross-validation results
print("\nCross-validation results:")
print(f"Mean Validation Loss: {np.mean(fold_losses):.4f} ± {np.std(fold_losses):.4f}")
print(f"Mean Validation Accuracy: {np.mean(fold_accuracies):.4f} ± {np.std(fold_accuracies):.4f}")

# Final Model Evaluation
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
print("Test Accuracy:", accuracy)
f1 = f1_score(y_test, y_pred_classes, average='weighted')
print("Test F1 Score:", f1)
print("\nClassification Report:\n", classification_report(y_test, y_pred_classes))



# Calculate the confusion matrix
conf_matrix = confusion_matrix(y_test_encoded, y_pred_classes)

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()


end_time = time.time()
elapsed_time = end_time - start_time
print(f"Execution time: {elapsed_time} seconds")

VariationalAutoencoder full

In [ ]:
start_time = time.time()


# Load the data
data = pd.read_csv("METABRIC_RNA_Mutation.csv", low_memory=False)

# Data Cleaning: Drop specified columns and encode target variable
columns_to_drop = [
    "age_at_diagnosis", "tumor_other_histologic_subtype", "inferred_menopausal_state",
    "mutation_count", "hormone_therapy", "patient_id", "cellularity",
    "type_of_breast_surgery", "cohort", "chemotherapy", "tumor_size",
    "hormone_therapy", "menopausal_state", "overall_survival_months",
    "overall_survival", "death_from_cancer", "tumor_stage", "er_status",
    "pam50_+_claudin-low_subtype", "pr_status", "er_status_measured_by_ihc",
    "nottingham_prognostic_index", "tumor_stage", "death_from_cancer",
    "integrative_cluster", "her2_status_measured_by_snp6", "her2_status",
    "radio_therapy", "3-gene_classifier_subtype", "oncotree_code"
]
data = data.drop(columns=columns_to_drop, errors='ignore')
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data["cancer_type_detailed"])

# One-Hot Encoding & Imputation
X = pd.get_dummies(data.drop(columns=["cancer_type_detailed"]), drop_first=True)
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Feature Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Feature Selection with Lasso
lasso = Lasso(alpha=0.01)
lasso.fit(X_scaled, y)
selected_featuresLasso = X.columns[(lasso.coef_ != 0)]
X_selected = X_scaled[:, lasso.coef_ != 0]

if X_selected.shape[1] == 0:
    print("Warning: Lasso selected 0 features. Using all features instead.")
    X_selected = X_scaled  # Use all features if Lasso selects none

# VAE for Dimensionality Reduction
input_dim = X_selected.shape[1]
latent_dim = 50  # Latent space dimension for dimensionality reduction

# Define VAE model
def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

# Custom layer for KL divergence loss
class KLDivergenceLayer(Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        self.add_loss(K.mean(kl_loss))
        return z_mean

# Encoder network
inputs = Input(shape=(input_dim,))
h = Dense(128, activation='relu')(inputs)
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)
z_mean = KLDivergenceLayer()([z_mean, z_log_var])
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# Decoder network
decoder_h = Dense(128, activation='relu')
decoder_mean = Dense(input_dim, activation='sigmoid')
h_decoded = decoder_h(z)
x_decoded_mean = decoder_mean(h_decoded)

# Define the VAE model
vae = Model(inputs, x_decoded_mean)

# Custom VAE loss function for reconstruction only
def vae_loss(inputs, x_decoded_mean):
    reconstruction_loss = K.sum(K.square(inputs - x_decoded_mean), axis=-1)
    return K.mean(reconstruction_loss)

# Compile the VAE model with the custom loss
vae.compile(optimizer='adam', loss=vae_loss)

# Train VAE
vae.fit(X_selected, X_selected, epochs=50, batch_size=32, shuffle=True, validation_split=0.2, verbose=0)

# Extract encoder part to get compressed latent space representation
encoder = Model(inputs, z_mean)
X_encoded = encoder.predict(X_selected)

# Impute NaN values in X_encoded using SimpleImputer
imputer = SimpleImputer(strategy='mean')  # or 'median', 'most_frequent'
X_encoded = imputer.fit_transform(X_encoded)

if X_encoded.shape[1] == 0:
    print("Warning: X_encoded has 0 features after imputation. Using original features instead.")
    X_encoded = X_selected  # Use original features if X_encoded is empty

# Class Filtering: Remove classes with fewer than 2 instances
class_counts = Counter(y)
single_instance_classes = [cls for cls, count in class_counts.items() if count < 2]
mask = ~np.isin(y, single_instance_classes)
X_filtered = X_encoded[mask]
y_filtered = y[mask]

# Data Splitting
X_train, X_test, y_train, y_test = train_test_split(X_filtered, y_filtered, test_size=0.3, stratify=y_filtered, random_state=42)

# Class Balancing with SMOTE
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Cross-Validation Training
num_folds = 5
batch_size = 32
epochs = 20
learning_rate = 0.0005
y_train_balanced_encoded = LabelEncoder().fit_transform(y_train_balanced)
input_shape = X_train_balanced.shape[1]
num_classes = len(np.unique(y_train_balanced_encoded))

kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
fold_accuracies = []
fold_losses = []

for fold, (train_idx, val_idx) in enumerate(kfold.split(X_train_balanced, y_train_balanced_encoded)):
    print(f"Training fold {fold+1}/{num_folds}...")
    X_train_fold, X_val_fold = X_train_balanced[train_idx], X_train_balanced[val_idx]
    y_train_fold, y_val_fold = y_train_balanced_encoded[train_idx], y_train_balanced_encoded[val_idx]

    model = Sequential([
        Dense(128, activation='relu', kernel_regularizer=l2(0.001), input_shape=(input_shape,)),
        Dropout(0.2),
        Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
        Dropout(0.2),
        Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
        Dropout(0.2),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train_fold, y_train_fold, epochs=epochs, batch_size=batch_size, validation_data=(X_val_fold, y_val_fold), verbose=1)
    val_loss, val_accuracy = model.evaluate(X_val_fold, y_val_fold, verbose=0)
    fold_losses.append(val_loss)
    fold_accuracies.append(val_accuracy)
    print(f"Fold {fold+1} - Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}")

# Report cross-validation results
print("\nCross-validation results:")
print(f"Mean Validation Loss: {np.mean(fold_losses):.4f} ± {np.std(fold_losses):.4f}")
print(f"Mean Validation Accuracy: {np.mean(fold_accuracies):.4f} ± {np.std(fold_accuracies):.4f}")

# Final Model Evaluation
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
print("Test Accuracy:", accuracy)
f1 = f1_score(y_test, y_pred_classes, average='weighted')
print("Test F1 Score:", f1)
print("\nClassification Report:\n", classification_report(y_test, y_pred_classes))


end_time = time.time()
elapsed_time = end_time - start_time
print(f"Execution time: {elapsed_time} seconds")

full code excepte replace SMOTE with GANS


In [ ]:
start_time = time.time()

# Load the data
data = pd.read_csv("METABRIC_RNA_Mutation.csv", low_memory=False)

# Data Cleaning: Drop specified columns and encode target variable
columns_to_drop = [
    "age_at_diagnosis", "tumor_other_histologic_subtype", "inferred_menopausal_state",
    "mutation_count", "hormone_therapy", "patient_id", "cellularity",
    "type_of_breast_surgery", "cohort", "chemotherapy", "tumor_size",
    "hormone_therapy", "menopausal_state", "overall_survival_months",
    "overall_survival", "death_from_cancer", "tumor_stage", "er_status",
    "pam50_+_claudin-low_subtype", "pr_status", "er_status_measured_by_ihc",
    "nottingham_prognostic_index", "tumor_stage", "death_from_cancer",
    "integrative_cluster", "her2_status_measured_by_snp6", "her2_status",
    "radio_therapy", "3-gene_classifier_subtype", "oncotree_code"
]
data = data.drop(columns=columns_to_drop, errors='ignore')
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data["cancer_type_detailed"])

# One-Hot Encoding & Imputation
X = pd.get_dummies(data.drop(columns=["cancer_type_detailed"]), drop_first=True)
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Feature Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Feature Selection with Lasso
lasso = Lasso(alpha=0.01)
lasso.fit(X_scaled, y)
selected_featuresLasso = X.columns[(lasso.coef_ != 0)]
X_selected = X_scaled[:, lasso.coef_ != 0]

# Autoencoder for Dimensionality Reduction
input_dim = X_selected.shape[1]
encoding_dim = 50

input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation='relu')(input_layer)
decoder = Dense(input_dim, activation='sigmoid')(encoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)
autoencoder.compile(optimizer='adam', loss='mse')

# Train the autoencoder
autoencoder.fit(X_selected, X_selected, epochs=50, batch_size=32, shuffle=True, validation_split=0.2, verbose=0)

# Extract the encoder part to get the compressed representation
encoder_model = Model(inputs=input_layer, outputs=encoder)
X_encoded = encoder_model.predict(X_selected)

# Class Filtering: Remove classes with fewer than 2 instances
class_counts = Counter(y)
single_instance_classes = [cls for cls, count in class_counts.items() if count < 2]
mask = ~np.isin(y, single_instance_classes)
X_filtered = X_encoded[mask]
y_filtered = y[mask]

print(X_filtered.shape)

# Data Splitting
X_train, X_test, y_train, y_test = train_test_split(X_filtered, y_filtered, test_size=0.3, stratify=y_filtered, random_state=42)

# Define the GAN
latent_dim = 100  # Size of the noise vector
num_features = X_train.shape[1]

def build_generator(latent_dim, num_features):
    # Create a LeakyReLU object
    leaky_relu = LeakyReLU(alpha=0.2)

    model = Sequential([
        Input(shape=(latent_dim,)),
        Dense(128, activation=leaky_relu),  # Use leaky_relu as activation
        Dense(256, activation=leaky_relu),  # Use leaky_relu as activation
        Dense(512, activation=leaky_relu),  # Use leaky_relu as activation
        Dense(num_features, activation='linear')
    ])
    return model

def build_discriminator(num_features):
    model = Sequential([
        Input(shape=(num_features,)),
        Dense(512),
        LeakyReLU(alpha=0.2),  # Changed 'negative_slope' to 'alpha'
        Dropout(0.3),
        Dense(256),
        LeakyReLU(alpha=0.2),  # Changed 'negative_slope' to 'alpha'
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0002, beta_1=0.5), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_gan(generator, discriminator):
    discriminator.trainable = False
    gan_input = Input(shape=(latent_dim,))
    generated_data = generator(gan_input)
    gan_output = discriminator(generated_data)
    gan = Model(gan_input, gan_output)
    gan.compile(optimizer=Adam(learning_rate=0.0002, beta_1=0.5), loss='binary_crossentropy')
    return gan

generator = build_generator(latent_dim, num_features)
discriminator = build_discriminator(num_features)
gan = build_gan(generator, discriminator)

def train_gan(generator, discriminator, gan, X_train_minority, epochs=100, batch_size=128):
    half_batch = batch_size // 2
    for epoch in range(epochs):
        idx = np.random.randint(0, X_train_minority.shape[0], half_batch)
        real_samples = X_train_minority[idx]
        real_labels = np.ones((half_batch, 1))
        d_loss_real = discriminator.train_on_batch(real_samples, real_labels)

        noise = np.random.normal(0, 1, (half_batch, latent_dim))
        fake_samples = generator.predict(noise)
        fake_labels = np.zeros((half_batch, 1))
        d_loss_fake = discriminator.train_on_batch(fake_samples, fake_labels)

        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        valid_labels = np.ones((batch_size, 1))
        g_loss = gan.train_on_batch(noise, valid_labels)

        d_loss_real_value = d_loss_real[0] if isinstance(d_loss_real, (list, tuple)) else d_loss_real
        d_loss_fake_value = d_loss_fake[0] if isinstance(d_loss_fake, (list, tuple)) else d_loss_fake
        g_loss_value = g_loss[0] if isinstance(g_loss, (list, tuple)) else g_loss

        if epoch % 10 == 0:
            print(f"{epoch}/{epochs} [D loss: {0.5 * (d_loss_real_value + d_loss_fake_value):.4f}] [G loss: {g_loss_value:.4f}]")

# Identify minority class
minority_class = 1  # Replace with actual minority class
X_train_minority = X_train[y_train == minority_class]

# Train the GAN
train_gan(generator, discriminator, gan, X_train_minority)

# Generate synthetic samples
num_synthetic_samples = X_train_minority.shape[0]
noise = np.random.normal(0, 1, (num_synthetic_samples, latent_dim))
synthetic_samples = generator.predict(noise)

# Combine synthetic samples with original training data
X_train_balanced = np.vstack((X_train, synthetic_samples))
y_train_balanced = np.hstack((y_train, [minority_class] * num_synthetic_samples))

# Check the shape of the dataset after adding synthetic samples
print("Shape of X_train_balanced:", X_train_balanced.shape)
print("Shape of y_train_balanced:", y_train_balanced.shape)

# Check the class distribution after GANs
class_distribution = Counter(y_train_balanced)
print("Class distribution after GANs:", class_distribution)


# Cross-validation Training
num_folds = 5
batch_size = 32
epochs = 20
learning_rate = 0.0005
y_train_balanced_encoded = LabelEncoder().fit_transform(y_train_balanced)
input_shape = X_train_balanced.shape[1]
num_classes = len(np.unique(y_train_balanced_encoded))

kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
fold_accuracies = []
fold_losses = []

for fold, (train_idx, val_idx) in enumerate(kfold.split(X_train_balanced, y_train_balanced_encoded)):
    print(f"Training fold {fold+1}/{num_folds}...")
    X_train_fold, X_val_fold = X_train_balanced[train_idx], X_train_balanced[val_idx]
    y_train_fold, y_val_fold = y_train_balanced_encoded[train_idx], y_train_balanced_encoded[val_idx]

    model = Sequential([
        Dense(128, activation='relu', kernel_regularizer=l2(0.001), input_shape=(input_shape,)),
        Dropout(0.2),
        Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
        Dropout(0.2),
        Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
        Dropout(0.2),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train_fold, y_train_fold, epochs=epochs, batch_size=batch_size, validation_data=(X_val_fold, y_val_fold), verbose=1)
    val_loss, val_accuracy = model.evaluate(X_val_fold, y_val_fold, verbose=0)
    fold_losses.append(val_loss)
    fold_accuracies.append(val_accuracy)
    print(f"Fold {fold+1} - Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}")

# Report cross-validation results
print("\nCross-validation results:")
print(f"Mean Validation Loss: {np.mean(fold_losses):.4f} ± {np.std(fold_losses):.4f}")
print(f"Mean Validation Accuracy: {np.mean(fold_accuracies):.4f} ± {np.std(fold_accuracies):.4f}")

# Final Model Evaluation
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
print("Test Accuracy:", accuracy)
f1 = f1_score(y_test, y_pred_classes, average='weighted')
print("Test F1 Score:", f1)
print("\nClassification Report:\n", classification_report(y_test, y_pred_classes))


# Check the shape of the test set
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)

# Check the class distribution in the test set
class_distribution_test = Counter(y_test)
print("Class distribution in y_test:", class_distribution_test)


# Calculate the confusion matrix
conf_matrix = confusion_matrix(y_test_encoded, y_pred_classes)

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Execution time: {elapsed_time} seconds")